In [1]:
import torch.nn as nn

from datasets import *
from evaluation import *
from models import *
from pruning.unstructured import *
from pruning.structured import *

c:\Users\prabh\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
c:\Users\prabh\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


Importing C:\Users\prabh\Smart-pruner-dev\pruning\unstructured\L1norm.py
Importing C:\Users\prabh\Smart-pruner-dev\pruning\Train.py
Importing C:\Users\prabh\Smart-pruner-dev\pruning\structured\decay_pruning.py
Importing C:\Users\prabh\Smart-pruner-dev\pruning\structured\two_to_four.py
Importing C:\Users\prabh\Smart-pruner-dev\pruning\structured\Thinet.py


In [2]:
dataset = MNIST()
train_loader, test_loader = dataset.get_dataloader()

In [3]:
#Thinet
original_model = Lenet()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(original_model.parameters(), lr=0.001)

trainer = Thinet(original_model, 5, train_loader, criterion, optimizer,amount=0.5)  #init amount = 0.2(no of channels)

#train and prune the model and return both the original and pruned model
pruned_model = trainer.prune_model()

Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
tensor([6.1648, 6.5419, 5.9628, 5.9906, 6.6169, 5.7836, 5.6063, 6.0727, 6.5079,
        5.9620, 6.0269, 6.5983, 6.2738, 5.9894, 6.1441, 6.1783])
16
8
tensor([ 6,  5,  9,  2, 13,  3, 10,  7])
Pruned conv2 layer


AttributeError: 'Conv2d' object has no attribute 'input_height'

In [ ]:
for i ,j in pruned_model.named_parameters():
    print(i,j.shape)
    

NameError: name 'pruned_model' is not defined

In [5]:
for i ,j in original_model.named_parameters():
    print(i,j.shape)
    

conv1.weight torch.Size([6, 1, 5, 5])
conv1.bias torch.Size([6])
conv2.weight torch.Size([16, 6, 5, 5])
conv2.bias torch.Size([16])
fc1.weight torch.Size([120, 256])
fc1.bias torch.Size([120])
fc2.weight torch.Size([84, 120])
fc2.bias torch.Size([84])
fc3.weight torch.Size([10, 84])
fc3.bias torch.Size([10])


In [22]:
for i,j in pruned_model.named_modules():
    print(i)
    print(j)
    print("\n")



Lenet(
  (conv1): Conv2d(1, 3, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(3, 8, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=128, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


conv1
Conv2d(1, 3, kernel_size=(5, 5), stride=(1, 1))


conv2
Conv2d(3, 8, kernel_size=(5, 5), stride=(1, 1))


fc1
Linear(in_features=128, out_features=120, bias=True)


fc2
Linear(in_features=120, out_features=84, bias=True)


fc3
Linear(in_features=84, out_features=10, bias=True)




In [3]:
#for UnstructuredL1normPrune

model = Lenet()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

trainer = UnstructuredL1normPrune(model, 5, train_loader, criterion, optimizer, 0.5)

#train and prune the model and return both the original and pruned model
original_model, pruned_model = trainer.train_prune_retrain()


  0%|          | 0/5 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [4]:
print("Accuracy of original model : ", accuracy(original_model, test_loader))
print("Accuracy of pruned model : ", accuracy(pruned_model, test_loader))
print("Number of non zero weights in original model : ", non_zero_weights(original_model))
print("Number of non zero weights in pruned model : ", non_zero_weights(pruned_model))
print("Compression Ratio : ", compression_ratio(original_model, pruned_model))

Accuracy of original model :  [98.47]
Accuracy of pruned model :  [97.83]
Number of non zero weights in original model :  44190
Number of non zero weights in pruned model :  22095
Compression Ratio :  0.5


In [6]:
#for Decay pruning with reverse=False
model = Lenet()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

trainer = DecayPrune(model, 5, train_loader, criterion, optimizer, 0.8, 0.05,
                     False)  #init pruning rate = 0.5, decay = 0.1

#train and prune the model and return both the original and pruned model
original_model, pruned_model = trainer.train_prune_retrain()

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 4.3614 | 
Epoch: 2 | train_loss: 2.9750 | 
Epoch: 3 | train_loss: 2.4691 | 
Epoch: 4 | train_loss: 2.0970 | 
Epoch: 5 | train_loss: 1.7896 | 
Training is done
Pruning is done


In [7]:
print("Accuracy of original model : ", accuracy(original_model, test_loader))
print("Accuracy of pruned model : ", accuracy(pruned_model, test_loader))
print("Number of non zero weights in original model : ", non_zero_weights(original_model))
print("Number of non zero weights in pruned model : ", non_zero_weights(pruned_model))
print("Compression Ratio : ", compression_ratio(original_model, pruned_model))

Accuracy of original model :  [98.43]
Accuracy of pruned model :  [88.53]
Number of non zero weights in original model :  44190
Number of non zero weights in pruned model :  18582
Compression Ratio :  0.42050237610319074


In [ ]:
#for Decay pruning with reverse=True
model = AlexNet()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

trainer = DecayPrune(model, 5, train_loader, criterion, optimizer, 0.8, 0.05,
                     True)  #init pruning rate = 0.5, decay = 0.1

#train and prune the model and return both the original and pruned model
original_model, pruned_model = trainer.train_prune_retrain()

In [ ]:
print("Accuracy of original model : ", accuracy(original_model, test_loader))
print("Accuracy of pruned model : ", accuracy(pruned_model, test_loader))
print("Number of non zero weights in original model : ", non_zero_weights(original_model))
print("Number of non zero weights in pruned model : ", non_zero_weights(pruned_model))
print("Compression Ratio : ", compression_ratio(original_model, pruned_model))

In [6]:
print("Accuracy of original model : ", accuracy(original_model, test_loader))
print("Accuracy of pruned model : ", accuracy(pruned_model, test_loader))
print("Number of non zero weights in original model : ", non_zero_weights(original_model))
print("Number of non zero weights in pruned model : ", non_zero_weights(pruned_model))
print("Compression Ratio : ", compression_ratio(original_model, pruned_model))

Accuracy of original model :  [8.74]


RuntimeError: mat1 and mat2 shapes cannot be multiplied (128x128 and 256x120)